In [379]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import category_encoders as ce

In [341]:
df = pd.read_csv('hotels.csv')
df.head()
# df.info()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


In [342]:
columns = df.columns.to_list()
num_cols = list()
cat_cols = list()

for column in columns:
    if df[column].dtype == 'O':
        cat_cols.append(column)
    else:
        num_cols.append(column)

len(cat_cols)
len(num_cols)

9

In [344]:
feature_df = pd.DataFrame()
df['hotel_name'].nunique()
df['review_date'] = pd.to_datetime(df['review_date'])
df['review_date'].max()

n_hotels = df.copy()

def rev_func(num_tags):
    num_tags = num_tags[2:-2]
    res = num_tags.strip().split(' \', \' ')
    return res

n_hotels['tags_n'] = n_hotels['tags'].apply(rev_func)


n_hotels = n_hotels.explode('tags_n')

In [345]:
n_hotels['tags_n'].sort_values(ascending = False)
n_hotels.groupby('tags_n')['hotel_name'].count().sort_values(ascending=False)

tags_n
Leisure trip                                        313593
Submitted from a mobile device                      230778
Couple                                              189212
Stayed 1 night                                      145373
Stayed 2 nights                                     100263
                                                     ...  
Courtyard Suite with St Stephen s Cathedral view         1
Executive King Suite with Lounge Access                  1
Couture Suite                                            1
Octagon Suite with Lounge Access                         1
Studio My Little Haussmann                               1
Name: hotel_name, Length: 2368, dtype: int64

In [346]:
# sns.heatmap(df.corr(numeric_only = True), annot=True)

In [368]:
def get_country(address):
    country = address.split(' ')[-2:]
    country = str(country).replace(',', '').replace("['", "").replace("']", "").replace("'", "")
    if country == 'United Kingdom':
        return 'United Kingdom'
    if country == 'Paris France':
        return 'France'
    if country == 'Amsterdam Netherlands':
        return 'Amsterdam'
    if country == 'Milan Italy':
        return 'Italy'
    if country == 'Vienna Austria':
        return 'Austria'
    if country == 'Barcelona Spain':
        return 'Spain'
    else:
        return 'other'
    
df['country'] = df['hotel_address'].apply(get_country)

df['country'].unique()


def if_capital(country, address):
    if country == 'United Kingdom':
        if 'London' in address:
            return 1
        else:
            return 0
    if country == 'France':
        if 'Paris' in address:
            return 1
        else:
            return 0
    if country == 'Amsterdam':
        if 'Netherlands' in address:
            return 1
        else:
            return 0
    if country == 'Italy':
        if 'Milan' in address:
            return 1
        else:
            return 0
    if country == 'Austria':
        if 'Vienna' in address:
            return 1
        else:
            return 0
    if country == 'Spain':
        if 'Barcelona' in address:
            return 1
        else:
            return 0
        
df['if_capital'] = df.apply(lambda row: if_capital(row['country'], row['hotel_address']), axis = 1)

df = df.drop('hotel_address', axis = 1)

url = 'https://www.belastingdienst.nl/wps/wcm/connect/bldcontenten/belastingdienst/individuals/tax-regulations/tax_treaties/list-of-eu-countries/'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
eu_countries = page.find('article').find_all('li')
eu_countries = list(eu_countries)

for i in range(len(eu_countries)):
    eu_countries[i] = (str(eu_countries[i])).replace('<li>','').replace('</li>', '').replace('\xa0', ' ').replace('*', '').replace("'", "")
eu_countries


In [378]:
# df['review_date'] = df['review_date'].dt.year
df['reviewer_nationality'].nunique()

225

In [381]:
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) 
type_bin = bin_encoder.fit_transform(df['reviewer_nationality'])
df = pd.concat([df, type_bin], axis=1)

df

,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,...,country,if_capital,reviewer_nationality_0,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7
0,581,2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,...,United Kingdom,1,0,0,0,0,0,0,0,1
1,299,2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,...,United Kingdom,1,0,0,0,0,0,0,0,1
2,32,2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,...,France,1,0,0,0,0,0,0,1,0
3,34,2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,...,France,1,0,0,0,0,0,0,0,1
4,914,2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,...,Amsterdam,1,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386798,107,2017,9.0,Hotel Moonlight,France,No Negative,0,617,Tr s proche du metro Earl s court,10,...,United Kingdom,1,0,0,0,0,0,1,0,0
386799,272,2017,8.4,BEST WESTERN PLUS Amedia Wien,Turkey,No Negative,0,3224,The bed was so comfy I stayed with my boyfrie...,93,...,Austria,1,0,0,1,0,1,0,1,1
386800,457,2016,6.8,Bloomsbury Palace Hotel,Netherlands,room is really small but guess is normal in L...,12,2751,great location simple check in out nice shower,9,...,United Kingdom,1,0,0,0,0,1,1,1,1
386801,365,2017,8.1,The Marble Arch London,United Arab Emirates,No Negative,0,1567,Location and very comfy bed,6,...,United Kingdom,1,0,0,0,0,1,0,0,0


СТРОКОВЫЕ ДАННЫЕ

В исходном наборе данных всего девять столбцов содержат числовые данные. Так что просто удалить все object-значения и считать задачу выполненной не получится. Мы сделали это в прошлом юните, чтобы получить значение метрики. Но чтобы улучшить качество модели, вам перед удалением придётся очень основательно поработать со строковыми данными и извлечь из них как можно больше информации, которую можно представить в числовом виде.

ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ

Мы уже говорили о том, что в задачах по машинному обучению принято не удалять строки с пустыми значениями, а заполнять их максимально близкими к реальности данными. Используйте знания из модулей очистки данных или вспомните о мере центральной тенденции в юнитах статистики. Попробуйте различные способы и ориентируйтесь на итоговую метрику.

НОВЫЕ ПРИЗНАКИ

Это, пожалуй, самая творческая часть работы на данном шаге. Создание новых признаков потребует от вас способности вникать в контент, умения подключать к работе интуицию, творческого подхода и готовности к экспериментам. Попробуйте извлечь несколько признаков из даты, проанализируйте датасет на возможность добавления внешних источников данных. При наличии времени разберите текстовые столбцы на признаки.

ПРЕОБРАЗОВАНИЕ ПРИЗНАКОВ

Данные могут быть представлены в невыгодном для модели свете. Используйте изученные вами способы преобразования данных, чтобы сделать данные качественнее.

ОТБОР ПРИЗНАКОВ

Признаки могут дублировать информацию. Анализ мультиколлинеарности мы будем использовать, чтобы выбрать уникальные признаки для модели.

Также перед обучением (после разделения выборки на X и y) мы можем оценить значимость каждого признака для целевой переменной — рейтинг отеля. При помощи теста мы сможем узнать, как географическая долгота отеля и количество негативных отзывов влияют на его рейтинг.

Для оценки значимости категориальных признаков будем использовать тест хи-квадрат, для непрерывных признаков — тест ANOVA.